In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [37]:
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image
import numpy as np
from path import Path as path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score

In [3]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

d:\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [18]:
imagenet = models.resnet34(pretrained=True)

In [5]:
def PIL_to_raw(img_pil):
    img_tensor = preprocess(img_pil)
    img_tensor.unsqueeze_(0)
    img_variable = Variable(img_tensor)
    fc_out = imagenet(img_variable)
    return fc_out.data.numpy()

def pred_vector(img_pil):
    raw = PIL_to_raw(img_pil)
    preds = raw[0]
    result = np.array([np.exp(p) for p in preds])
    return result / sum(result)

In [10]:
mydir = "../../data/dogscats"

In [12]:
cat = 0
dog = 1

In [13]:
X_train, y_train, X_test, y_test = [],[],[],[]

In [19]:
for f in path(mydir + "/train/cats").files():
    X_train.append(pred_vector(Image.open(f)))
    y_train.append(cat)

In [20]:
for f in path(mydir + "/train/dogs").files():
    X_train.append(pred_vector(Image.open(f)))
    y_train.append(dog)

In [22]:
for f in path(mydir + "/valid/cats").files():
    X_test.append(pred_vector(Image.open(f)))
    y_test.append(cat)

In [23]:
for f in path(mydir + "/valid/dogs").files():
    X_test.append(pred_vector(Image.open(f)))
    y_test.append(dog)

In [24]:
X_train = np.array(X_train)

In [25]:
X_train.shape

(23000, 1000)

In [26]:
y_train = np.array(y_train)

In [27]:
y_train.shape

(23000,)

In [28]:
X_trest = np.array(X_test)

In [29]:
X_test = X_trest

In [30]:
y_test = np.array(y_test)

In [31]:
rf = RandomForestClassifier(n_estimators=50)

In [32]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
y_pred = rf.predict(X_test)

In [35]:
f1_score(y_test, y_pred)

0.9278870398386284

In [38]:
accuracy_score(y_test, y_pred)

0.9285